In [1]:
import sys
assert sys.version_info >= (3, 5)

import re,os,csv
import numpy as np
import pandas as pd
from unidecode import unidecode
import dedupe

C:\Users\shitiza\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\shitiza\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\shitiza\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


#### Pre-Prcoess & Clean-up Data Sets before entity Matching

In [70]:
def clean_objCols(dfx):
    for column in dfx.columns.to_list() :
        if np.dtype('O') == dfx[column].dtype:
            #print(f'INFO: clean-up col : {column}')
            dfx[column] = (dfx[column]
                .str.replace(r'\[|\]-|,|;|;|"|\'','', regex=True,flags=re.IGNORECASE)
                .str.replace(r'\s+|/',' ', regex=True,flags=re.IGNORECASE)
                .str.strip()
                .str.strip('"')
                .str.strip("'")
                .str.strip('\[')
                .str.strip('\]')
                .str.lower()
                .str.strip()
                )
    return dfx

def cleanup_cols(dfx,outF,maxL=1000):
    return (
    clean_objCols(dfx)
    #.assign(id = lambda df : df.id.astype(int))
    .assign(id = lambda df : df.index)
    .assign(fileName  = lambda df : [f + '__' + str(i) for f,i in zip(df.fileName,df.index)])
    .loc[:maxL,['fileName','id','Title','Keywords']]
    .set_index('fileName')
    .fillna('')
    .to_csv(outF)
    )

#### Read left & right datasets

In [125]:
DATA_DIR = 'CONSOLIDATED_FOR_ENTITY_MATCHING'
   
OUT_DIR = 'ENTITY_MATCH_OUTPUT/pubMED_vs_nih'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)


o_left_file = DATA_DIR + '/all_pubmed_keyWords_FINAL.csv'
o_right_file = DATA_DIR + '/all_nih_keyWords_FINAL.csv'

left_file = OUT_DIR + '/pubMedF.csv'
right_file = OUT_DIR + '/nihF.csv'
cleanup_cols(pd.read_csv(o_left_file).assign(fileName = 'pubMedF').astype(str),left_file)
cleanup_cols(pd.read_csv(o_right_file).assign(fileName = 'nihF').astype(str).rename(columns={'PROJECT_TITLE': 'Title'}),right_file)

In [126]:
pubMed_df = pd.read_csv(left_file).set_index('fileName') 
pubMed_df.head(4)

,id,Title,Keywords
fileName,,,
pubmedf__0,0,micra® leadless pacemaker,leadless presenting approach pacemaker datum ®...
pubmedf__1,1,pacemaker complications and costs: a nationwid...,leadless study novel may system datum pmsi sys...
pubmedf__2,2,runaway pacemaker,examination datum implantation revealed excell...
pubmedf__3,3,pacemaker reprogramming rarely needed after de...,action depending needed involved subsequent 17...


In [127]:
clinic_df = pd.read_csv(right_file).set_index('fileName')
clinic_df.head(4)

,id,Title,Keywords
fileName,,,
nihf__0,0,innovating organ shipment by studying environm...,vascular cell adhesion molecule intercellular ...
nihf__1,1,role of the soluble (pro)renin receptor in blo...,blood pressure reduction salt sensitive hypert...
nihf__2,2,hypertension augmented covid-19 through renin-...,severe acute respiratory syndrome mass spectru...
nihf__3,3,developing a new drug for treating myocardial ...,small business innovation research grant opera...


In [128]:
data_1 = pubMed_df.to_dict('index')
data_2 = clinic_df.to_dict('index')

In [129]:
data_1['pubmedf__0']


{'id': 0,
 'Title': 'micra® leadless pacemaker',
 'Keywords': 'leadless presenting approach pacemaker datum ® atrial similar traditional pacemakercardiac method therapy technology without miniature expectancy risk providesmicra invasive less high conclusion conventional transvenous crt versus year fibrillation therefore micra patients venous pacemakers device life'}

In [131]:
data_2['nihf__0']

{'id': 0,
 'Title': 'innovating organ shipment by studying environmental factors which affect organs during cold preservation',
 'Keywords': 'vascular cell adhesion molecule intercellular adhesion molecule 1 cell adhesion molecules environmental risk factor cell adhesion cell death environmental impact vertebral column transplant model survival ratestudying environmental factors innovating organ shipment cold preservation affect organs'}

In [132]:
output_file = OUT_DIR + '/data_matching_output.csv'
settings_file = OUT_DIR + '/data_matching_learned_settings'
training_file = OUT_DIR + '/data_matching_training_f.json'
def descriptions():
        for dataset in (data_1, data_2):
            for record in dataset.values():
                yield record['Title']   
def brands():
        for dataset in (data_1, data_2):
            for record in dataset.values():
                yield record['Keywords']   

In [133]:
    if os.path.exists(settings_file):
        print('reading from', settings_file)
        with open(settings_file, 'rb') as sf:
            linker = dedupe.StaticRecordLink(sf)

    else:
        # Define the fields the linker will pay attention to
        #
        # Notice how we are telling the linker to use a custom field comparator
        # for the 'price' field.
        fields = [
            {'field': 'Title', 'type': 'String'},
            {'field': 'Title', 'type': 'Text', 'corpus': descriptions()},
            {'field': 'Keywords', 'type': 'Text',
             'has missing': True, 'corpus': brands()}
            ]

        # Create a new linker object and pass our data model to it.
        linker = dedupe.RecordLink(fields)

        # If we have training data saved from a previous run of linker,
        # look for it an load it in.
        # __Note:__ if you want to train from scratch, delete the training_file
        if os.path.exists(training_file):
            print('reading labeled examples from ', training_file)
            with open(training_file) as tf:
                linker.prepare_training(data_1,
                                        data_2,
                                        training_file=tf,
                                        sample_size=15000)
        else:
            linker.prepare_training(data_1, data_2, sample_size=15000)

        # ## Active learning
        # Dedupe will find the next pair of records
        # it is least certain about and ask you to label them as matches
        # or not.
        # use 'y', 'n' and 'u' keys to flag duplicates
        # press 'f' when you are finished
        print('starting active labeling...')

        dedupe.console_label(linker)

        linker.train()

        # When finished, save our training away to disk
        with open(training_file, 'w') as tf:
            linker.write_training(tf)

        # Save our weights and predicates to disk.  If the settings file
        # exists, we will skip all the training and learning next time we run
        # this file.
        with open(settings_file, 'wb') as sf:
            linker.write_settings(sf)


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (sortedAcronym, Title)


starting active labeling...


Title : project my heart your heart: an idea whose time has come
Keywords : pacemaker cardiac effective ablation patients low pilot block including making methods recyclingidea every ventricular pacemaker throughout facilities years fibrillation ago 1 time studies optimal die define among charitable fully nearly several inception countries project year comeatrial idea year whose recycling servicing strategies heart practice small income patients 4 million case

Title : project 2
Keywords : coronary heart disease heart diseases disease rate tobacco use pharmaceutical preparations malignant neoplasms human disease cancer patient alcohol consumption data setproject 2

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


 n


Title : sinus bradycardia junctional rhythm and low-rate atrial fibrillation in short qt syndrome during 20 years of follow-up: three faces of the same genetic problem
Keywords : evolution genetic aortic dysfunction recurred implantation atrial three outcome kcnq1 mutation group geneshort followvalve risk junctional fibrillation 20 qt low high node v141m rate years sinus bradycardia infancy problem short requiring single fibrillation year faces tavr mortality progressive syndrome rhythm interval

Title : icecan
Keywords : united states national institutes midbrain central gray substance autonomic nervous system disorders autonomic nervous system congenital neurologic anomalies longitudinal prospective study heart rate variability public health relevance nervous system physiology chronic pelvic painicecan

0/10 positive, 1/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

Title : effect of peer support on health outcomes in patients with cardiac pacemaker implantation: a randomized controlled trial
Keywords : program randomized cardiac based patients datum implantation received trial method pacemaker group therapy telephone support risk self health routine controlled high conclusion conventional effectcardiac nurse provided among support crt care measures groupcardiac year care improves education outcomes peer usual device group

Title : randomized controlled trial of group prevention coaching
Keywords : health care facility randomized controlled trials cardiovascular risk factor cardiovascular disorder risk cardiovascular disorder prevention primary health care heart disease risk cardiovascular system cardiovascular diseases risk reductionrandomized controlled trial group prevention coaching

8/10 positive, 8/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 y


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (commonSixGram, Title)
Title : wireless power transfer for a pacemaker application
Keywords : contracting transfer system impulses datum charging electrical uses rechargeable increased method pacemaker electrodes therapy non risk pacemakers years power artificial high conclusion muscles regular batteries device crt device efficient year heart small pacemakerwireless medical many applicationcardiac

Title : wireless physiologic monitoring in postpartum women
Keywords : oriented research career development award united states national institutes vaginal delivery procedure world health organization intensive care units health care facility heart rate monitor clinical trials design heart rate dialysis procedurewireless physiologic monitoring postpartum women

9/10 positive, 8/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 f


Finished labeling
INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
C:\Users\shitiza\Anaconda3\lib\site-packages\rlr\crossvalidation.py:117: RuntimeWarning: invalid value encountered in double_scalars
  matthews_cc = ((true_dupes * true_distinct
INFO:rlr.crossvalidation:optimum alpha: 0.010000, score 0.1432094871570064
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (commonSixGram, Title), SimplePredicate: (tokenFieldPredicate, Title))


In [134]:
    # ## Blocking

    # ## Clustering

    # Find the threshold that will maximize a weighted average of our
    # precision and recall.  When we set the recall weight to 2, we are
    # saying we care twice as much about recall as we do precision.
    #
    # If we had more data, we would not pass in all the blocked data into
    # this function but a representative sample.

    print('clustering...')
    linked_records = linker.join(data_1, data_2, 0.0,'one-to-one')

    print('# duplicate sets', len(linked_records))
    # ## Writing Results

    # Write our original data back out to a CSV with a new column called
    # 'Cluster ID' which indicates which records refer to each other.

    cluster_membership = {}
    for cluster_id, (cluster, score) in enumerate(linked_records):
        for record_id in cluster:
            cluster_membership[record_id] = {'Cluster ID': cluster_id,
                                             'Link Score': score}

    with open(output_file, 'w', encoding='utf8') as f:

        header_unwritten = True

        for fileno, filename in enumerate((left_file, right_file)):
            with open(filename, 'r', encoding="utf8") as f_input:
                reader = csv.DictReader(f_input)

                if header_unwritten:

                    fieldnames = (['Cluster ID', 'Link Score', 'source file'] +
                                  reader.fieldnames)

                    writer = csv.DictWriter(f, fieldnames=fieldnames)
                    writer.writeheader()

                    header_unwritten = False

                for row_id, row in enumerate(reader):

                    record_id = str(re.sub(OUT_DIR,'',re.sub('\.\/|F\.csv','',os.path.basename(filename).lower()))).replace('.csv','') + '__' + str(row_id)
                    #print(record_id)
                              
                    cluster_details = cluster_membership.get(record_id, {})
                    row['source file'] = fileno
                    row.update(cluster_details)
                   

                    writer.writerow(row)

clustering...
# duplicate sets 906


#### Sanity check to ensure Cluster-IN & Score in final matched output csv

In [135]:
chk_df = pd.read_csv(output_file)
len(chk_df['Cluster ID'].unique())
g_m = chk_df['Cluster ID'] != np.nan
chk_df[g_m].describe()

,Cluster ID,Link Score,source file,id
count,1812.000000,1812.000000,2002.000000,2002.000000
mean,452.500000,0.465639,0.500000,500.000000
std,261.611711,0.102928,0.500125,289.035861
min,0.000000,0.238105,0.000000,0.000000
25%,226.000000,0.388744,0.000000,250.000000
50%,452.500000,0.464148,0.500000,500.000000
75%,679.000000,0.531623,1.000000,750.000000
max,905.000000,0.870118,1.000000,1000.000000


#### Use 'Link Score' to filter out matches

In [153]:
match_mask = chk_df['Link Score'] >= 0.55
chk_df_filtered = chk_df[match_mask]
chk_df_filtered.to_csv(OUT_DIR + '/data_matching_output_FILTERED_FINAL.csv',index=False)
print(f'Total matches with score > 0.55 : {chk_df_filtered.shape[0]}')

Total matches with score > 0.55 : 364


In [154]:
chk_df_filtered.loc[~chk_df_filtered['Cluster ID'].isna()].sort_values(by=['Cluster ID']).head(6)

,Cluster ID,Link Score,source file,fileName,id,Title,Keywords
816,0.0,0.870118,0,pubmedf__816,816,prior pacemaker implantation and clinical outc...,estimated may filtration cardiac dysfunction ....
1965,0.0,0.870118,1,nihf__964,964,quantitative mri for characterizing heart fail...,psychology ) coronary fibrosis magnetic resona...
706,1.0,0.811758,0,pubmedf__706,706,sex differences and long-term outcome in patie...,valvular underwent pacemaker cardiac ablation ...
1320,1.0,0.811758,1,nihf__319,319,androgen receptors and sex differences in the ...,enterobacteria phage p1 cre recombinase design...
493,2.0,0.797195,0,pubmedf__493,493,predictors of death in chronic chagas cardiomy...,study independent 150ms pacemaker fraction cla...
1427,2.0,0.797195,1,nihf__426,426,predictors of cardiomyopathy progression in a ...,connective tissue growth factor transforming g...


In [155]:
def agg_count(s):
    return len([v for v in s.unique()])
def agg_list(s):
    return [v for v in s.unique()]

#### Final Matched output with mapped ids Only

In [156]:
dump_df = chk_df_filtered.groupby(by = ['Cluster ID'])['id'].agg([agg_list]).reset_index()
dump_df['pubMed'] = np.array(dump_df.agg_list.to_list())[:,0]
dump_df['clinicTrial'] = np.array(dump_df.agg_list.to_list())[:,1]
dump_df.loc[:,['pubMed','clinicTrial']].to_csv(OUT_DIR + '/data_matching_Project_2_columnsOnly.csv',index=False)


In [157]:
def join_keywords(k1,k2):
    s1 = set(k1.split(' '))
    s2 = set(k2.split(' '))
    return ' '.join(list(s1.intersection(s2)))

merge_CnP_df = dump_df.loc[:,['pubMed','clinicTrial']]
out_merge_CnP_df = (merge_CnP_df
 .merge(pubMed_df,left_on=['pubMed'],right_on=['id']).rename(columns={'id':'pubMed_id','Title':'pubMed_Title','Keywords':'pubMed_keywords'})
 .merge(clinic_df,left_on=['clinicTrial'],right_on=['id']).rename(columns={'id':'clinicTrial_id','Title':'clinicTrial_Title','Keywords':'clinicTrial_keywords'})
 .assign(keywords_common = lambda df : df.apply(lambda x: join_keywords(x.pubMed_keywords, x.clinicTrial_keywords), axis=1) )
 .drop(['pubMed_id','clinicTrial_id'],axis=1)
)

print(f' No common keywords found for {out_merge_CnP_df.keywords_common.isna().sum()} ')
out_merge_CnP_df.to_csv(OUT_DIR + '/data_matching_Project_IMP_columnsOnly.csv')
out_merge_CnP_df.head(5)

 No common keywords found for 0 


,pubMed,clinicTrial,pubMed_Title,pubMed_keywords,clinicTrial_Title,clinicTrial_keywords,keywords_common
0,816,964,prior pacemaker implantation and clinical outc...,estimated may filtration cardiac dysfunction ....,quantitative mri for characterizing heart fail...,psychology ) coronary fibrosis magnetic resona...,ejection failure heart fraction clinical )
1,706,319,sex differences and long-term outcome in patie...,valvular underwent pacemaker cardiac ablation ...,androgen receptors and sex differences in the ...,enterobacteria phage p1 cre recombinase design...,) differences
2,493,426,predictors of death in chronic chagas cardiomy...,study independent 150ms pacemaker fraction cla...,predictors of cardiomyopathy progression in a ...,connective tissue growth factor transforming g...,disease cohort risk ) predictors cardiomyopathy
3,18,326,risk factor for cardiac permanent pacemaker in...,study square pacemaker regression cardiac pulm...,regulation and function of the circadian facto...,simian virus 40 public health relevance high f...,
4,334,519,lead-sparing tricuspid valve repair damaged by...,may muscle pacemaker cardiac ablation block in...,removal of damaged mitochondria by alternative...,reactive oxygen species ripk1 gene release fac...,


#### Sample output to check matched entitties

In [158]:
dump_dfT=chk_df.groupby(by = ['Cluster ID'])['Title'].agg([agg_list]).reset_index()
np.array(dump_dfT.agg_list.to_list())[:2]

array([['prior pacemaker implantation and clinical outcomes in patients with heart failure and preserved ejection fraction',
        'quantitative mri for characterizing heart failure with preserved ejection fraction'],
       ['sex differences and long-term outcome in patients with pacemakers',
        'androgen receptors and sex differences in the biological clock']],
      dtype='<U224')

#### We need to create a ground-truth table with few enteries to verify that dedupe is matching as expected
#### Read ground-Truth Table to check prec/recall only. 

In [159]:
gnd_truth_df = pd.read_csv(OUT_DIR + '/gndTable_ProjectF.csv')

In [160]:
out_df = pd.read_csv(OUT_DIR + '/data_matching_Project_2_columnsOnly.csv')

In [161]:
def agg_count(s):
    return len([v for v in s.unique()])
def agg_list(s):
    return [v for v in s.unique()]

def evaluateDuplicates(found_dupes, true_dupes):
    true_positives = found_dupes.intersection(true_dupes)
    false_positives = found_dupes.difference(true_dupes)
    uncovered_dupes = true_dupes.difference(found_dupes)

    print('found duplicate')
    print(len(found_dupes))
    
    print(f'true_positives : {len(true_positives)}, false_positives : {len(false_positives)}, true_dupes : {len(true_dupes)}')

    prec = len(true_positives) / float(len(found_dupes))
    print(f'precision : {prec}')    

    rec = len(true_positives) / float(len(true_dupes))
    print(f'recall : {rec}')
        
    F1 = 2 * (prec * rec) / (prec + rec)
    print(f'F1 score : {F1}')


In [162]:
true_dupes = set(map(tuple, np.array(gnd_truth_df).tolist()))
found_dupes = set(map(tuple, np.array(out_df).tolist()))

In [163]:
evaluateDuplicates(found_dupes,true_dupes)

found duplicate
182
true_positives : 20, false_positives : 162, true_dupes : 20
precision : 0.10989010989010989
recall : 1.0
F1 score : 0.198019801980198


#### Trick to create ground-truth Table

In [144]:
gnd_mask = chk_df_filtered['Link Score'] >= 0.7
dump_df = chk_df_filtered[gnd_mask].groupby(by = ['Cluster ID'])['id'].agg([agg_list]).reset_index()
dump_df['pubMed'] = np.array(dump_df.agg_list.to_list())[:,0]
dump_df['clinicTrial'] = np.array(dump_df.agg_list.to_list())[:,1]
dump_df.loc[:,['pubMed','clinicTrial']].to_csv(OUT_DIR + '/gndTable_ProjectF.csv',index=False)